# How to crack a password using Gentic Algorithm?

Source: https://blog.sicara.com/getting-started-genetic-algorithms-python-tutorial-81ffa1dd72f9

In [19]:
import numpy as np
import string
import operator

### Choosing a fitness function

The evaluation function is the first step to create a genetic algorithm. It’s the function that estimates the success of our specimen: it will allow us to divide the population between the ugly duckling and the beautiful swans. The goal of this separation is that, later, the successful specimen will have more “chance” to get picked to form the next generation. 

What is our goal? Crack a password. Thus the goal of our function is to transform the binary result “fail or success” in a continuous mark from 0 (can’t fail more) to 100 (perfection).

The simplest solution here is:

$$fitness\ score = (number\ of\ char\ correct)\ /\ (total\ number\ of\ char)$$

That way, an individual with a bigger fitness result is a specimen genetically closer to success than the others. Thus the fitness function for our genetic algorithm will accurately sort the population.


In [6]:
def fitness(password, test_word):
    if (len(password) != len(test_word)):
        print('Incompatible')
        return
    else:
        score = 0
        for idx, char in enumerate(password):
            if (char == test_word[idx]):
                score += 1
    return score * 100 / len(password)

### Creating our individuals

So now we know how to evaluate our individuals; but how do we define them? This part is really tricky: the goal is to know what are the unalterable characteristics and what is variable.

The comparison with genetics is here really helpful. Indeed, the DNA is composed of genes, and each of those genes comes through different alleles (different versions of this gene). Genetic algorithms retain this concept of population’s DNA.

In our case, our individuals are going to be words (obviously of equal length with the password). Each letter is a gene and the value of the letter is the allele. In the word “banana”: ‘b’ is the allele of the first letter.

What is the point of this creation?

- We know that each of our individuals is keeping the good shape (a word with the correct size)
- Our population can cover every possibility (every word possible with this size).

Out genetic algorithm can then explore all possible combinations.

### Creating our first population

Now, we know what are the characteristics of our individuals and how we can evaluate their performance. We can now start the "evolution" step of our genetic algorithm.

The main idea to keep in mind when we create the first population is that we must not

point the population towards a solution that seems good. We must make the population as wide as possible and make it cover as many possibilities as possible. The perfect first population of a genetic algorithm should cover every existing allele.

So in our case, we are just going to create words only composed of random letters.

In [37]:
def generateWord(length):
    all_chars = list(string.ascii_letters + string.digits + \
                     '@' + '#' + '$' + '_')
    new_word = np.random.choice(all_chars, size=length, replace=True)
    new_word = "".join(new_word)
    return new_word

In [39]:
def generateFirstPopulation(population_size, password):
    population = []
    for i in range(population_size):
        population.append(generateWord(len(password)))
    return population

### From one generation to the next

Given a generation, in order to create the next one, we have 2 things to do. First we select a specific part of our current generation. Then the genetic algorithm combines those breeders in order to create the next batch.

#### Breeders selection

They are lots of way to do this but you must keep in mind two ideas: the goals are to select the best solutions of the previous generation and not to completely put aside the others. The hazard is: if you select only the good solutions at the beginning of the genetic algorithm you are going to converge really quickly towards a local minimum and not towards the best solution possible.

The solution to do that is to select, on the one hand, the N better specimen (N = best_sample) and on the other hand, to select M random individuals without distinction of fitness (M = lucky_few).

In [52]:
def computePopulationPerformance(population, password):
    popPerformance = {}
    for individual in population:
        popPerformance[individual] = fitness(individual, password)
    popPerformanceSorted = sorted(popPerformance, \
                                  key=operator.itemgetter(1), \
                                  reverse=True)
    return popPerformanceSorted[0]

In [88]:
def selectFromPopulation(popPerformanceSorted, best_sample, lucky_few):
    nextGeneration = popPerformanceSorted[0:best_sample]
    nextGeneration.extend(np.random.choice(popPerformanceSorted, lucky_few))
    np.random.shuffle(nextGeneration)
    return nextGeneration

### Breeding

We can keep on the biologic analogy for the breeding in our genetic algorithm. The goal of sexual reproduction is to mix the DNA of 2 individuals, so let’s do the same thing here. We have two individuals: “Tom” and “Jerry”, their DNA is defined by their alleles (the value of each letter). Thus in order to mix their DNA, we just have to mix their letters. There are lots of ways to do this so I picked the simplest solution: for each letter of the child, take randomly the letter of “Tom” or “Jerry”.

Obviously, the couple “Tom" and “Jerry” isn’t going to produce only one child. You have to fix the number of children per couple in order to keep a stable population in your genetic algorithm. The number of individuals in the generation 0 equals the number of individuals in the next generation.

In [100]:
def createChild(parent1, parent2):
    parents = list(parent1 + parent2)
    np.random.shuffle(parents)
    child = np.random.choice(parents, \
                             size=int((len(parent1)+len(parent2))/2), \
                             replace=False)
    child = ''.join(child)
    return child